Classes from BioRED corpus:
- 'CellLine'
- 'ChemicalEntity'
- 'DiseaseOrPhenotypicFeature'
- 'GeneOrGeneProduct'
- 'OrganismTaxon'
- 'SequenceVariant'

Importing libraries:

In [14]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import WordPunctTokenizer
import pickle

Converting files from .txt into  u neophodan oblik za treniranje:
- text vectors are separated sentences,
- in numeric vectors there are also those that consist only of zeros if they are not shorter than 3 tokens,
- here the presence of NER is recorded only by ones, not by twos (2=1).

In [2]:
# Training set

address = './Dataset/Train.PubTator' # Dataset is a folder which should contain downloaded Train, Dev and Test datasets

with open(address) as f:
    
    id_nums = list()
    ts = list()
    titles = list()
    abstracts = list()
    original_texts = list()
    texts = list()
    results = list()
    original_results = list()
    vectors = list()
    corpuses = list()
    sentences = list()
    original_sentences = list()
    
    for x in f.read().split('\n\n'):
        
        try:
            id_num = x.split('|t|')[0]
            if id_num in ['17000021', '17006606', '9294109', '20086182', '18310445', '28398555',
                         '30442153', '15184369', '16157158', '15198485', '17951029', '21666969',
                          '19825989', '21496008', '16321363', '16506214', '18791947', '20801104',
                          '28851297', '20651814', '17042910', '18340638', '19721134', '20683499']:
                continue
            else:
                id_num = id_num
            
            title = x.split('|t|')[1].split('\n')[0].rstrip()
            if title.endswith('?'):
                title = title[:-1] + '.'
            elif title.endswith('.'):
                title = title
            else:
                title = title + '.'
                
            abstract = x.split('|a|')[1].split('\n')[0].rstrip()
            if abstract.endswith('..'):
                abstract = abstract[:-1]
            elif abstract.endswith('.'):
                abstract = abstract
            else:
                abstract = abstract + '.'

            original_text = title + " " + abstract
            
            tags = list()
            for y in x.split('\n')[2:]:
                pocetni = y.split('\t')[1]
                if pocetni.isdigit():
                    tag = y.split('\t')[4]
                    tags.append(tag)
            
            for t in set(tags):
                text = original_text
                pomeraj = 0
                for y in x.split('\n')[2:]:
                    pocetni = y.split('\t')[1]
                    if pocetni.isdigit():
                        pocetni = int(pocetni) - pomeraj
                        krajnji = int(y.split('\t')[2]) - pomeraj
                        NER, tag = y.split('\t')[3:5]
                        
                        if tag == t:
                            NER_len = len(WordPunctTokenizer().tokenize(NER))
                            if NER_len == 1:
                                a = 'xxxxx'
                            else:
                                a = 'xxxxx'+' yyyyy'*(NER_len-1)
                        
                            text = text[:pocetni] + a + text[krajnji:]
                        
                            pomeraj += krajnji - pocetni - len(a)
                    else:
                        continue
                result = WordPunctTokenizer().tokenize(text)
                original_result = WordPunctTokenizer().tokenize(original_text)
                if len(result)==len(original_result):
                    pass
                else:
                    print(id_num, 'NIJE OK', len(result), len(original_result))
                
                n = 0
                for m in original_result:
                    if m == '.' or m == ').':
                        k = original_result.index(m, n)
                        sentence = result[n:k+1]
                        original_sentence = original_result[n:k+1]
                        n = k + 1
                        if len(sentence)==len(original_sentence):
                            pass
                        else:
                            print(id_num, 'NIJE OK', sentence, original_sentence)
                        vector = list()
                        for i in sentence:
                            if i=='xxxxx':
                                i=1
                            elif i=='yyyyy':
                                i=1
                            else:
                                i=0
                            vector.append(i)
                        corpus = 'BioRED'
                        if len(vector) >= 3:
                            id_nums.append(id_num)
                            ts.append(t)
                            titles.append(title)
                            abstracts.append(abstract)
                            original_texts.append(original_text)
                            texts.append(text)
                            results.append(result)
                            original_results.append(original_result)
                            corpuses.append(corpus)
                            sentences.append(sentence)
                            original_sentences.append(original_sentence)
                            vectors.append(vector)
        except:
            break

In [3]:
df_train = pd.DataFrame(list(zip(corpuses, id_nums, ts, original_sentences, vectors)),
               columns =['cor', 'ids', 'class', 'text', 'labels'])
df_train = df_train.set_index('ids')

In [4]:
# Development set

address = './Dataset/Dev.PubTator'

with open(address) as f:
    
    id_nums = list()
    ts = list()
    titles = list()
    abstracts = list()
    original_texts = list()
    texts = list()
    results = list()
    original_results = list()
    vectors = list()
    corpuses = list()
    sentences = list()
    original_sentences = list()
    
    for x in f.read().split('\n\n'):
        
        try:
            id_num = x.split('|t|')[0]
            if id_num in ['16867246', '19880293', '27103577', '17600377', '25589620', '20621845']:
                continue
            else:
                id_num = id_num
            
            title = x.split('|t|')[1].split('\n')[0].rstrip()
            if title.endswith('?'):
                title = title[:-1] + '.'
            elif title.endswith('.'):
                title = title
            else:
                title = title + '.'
                
            abstract = x.split('|a|')[1].split('\n')[0].rstrip()
            if abstract.endswith('..'):
                abstract = abstract[:-1]
            elif abstract.endswith('.'):
                abstract = abstract
            else:
                abstract = abstract + '.'

            original_text = title + " " + abstract
            
            tags = list()
            for y in x.split('\n')[2:]:
                pocetni = y.split('\t')[1]
                if pocetni.isdigit():
                    tag = y.split('\t')[4]
                    tags.append(tag)
            
            for t in set(tags):
                text = original_text
                pomeraj = 0
                for y in x.split('\n')[2:]:
                    pocetni = y.split('\t')[1]
                    if pocetni.isdigit():
                        pocetni = int(pocetni) - pomeraj
                        krajnji = int(y.split('\t')[2]) - pomeraj
                        NER, tag = y.split('\t')[3:5]
                        
                        if tag == t:
                            NER_len = len(WordPunctTokenizer().tokenize(NER))
                            if NER_len == 1:
                                a = 'xxxxx'
                            else:
                                a = 'xxxxx'+' yyyyy'*(NER_len-1)
                        
                            text = text[:pocetni] + a + text[krajnji:]
                        
                            pomeraj += krajnji - pocetni - len(a)
                    else:
                        continue
                result = WordPunctTokenizer().tokenize(text)
                original_result = WordPunctTokenizer().tokenize(original_text)
                if len(result)==len(original_result):
                    pass
                else:
                    print(id_num, 'NIJE OK', len(result), len(original_result))
                
                n = 0
                for m in original_result:
                    if m == '.' or m == ').':
                        k = original_result.index(m, n)
                        sentence = result[n:k+1]
                        original_sentence = original_result[n:k+1]
                        n = k + 1
                        if len(sentence)==len(original_sentence):
                            pass
                        else:
                            print(id_num, 'NIJE OK', sentence, original_sentence)
                        vector = list()
                        for i in sentence:
                            if i=='xxxxx':
                                i=1
                            elif i=='yyyyy':
                                i=1
                            else:
                                i=0
                            vector.append(i)
                        corpus = 'BioRED'
                        if len(vector) >= 3:
                            id_nums.append(id_num)
                            ts.append(t)
                            titles.append(title)
                            abstracts.append(abstract)
                            original_texts.append(original_text)
                            texts.append(text)
                            results.append(result)
                            original_results.append(original_result)
                            corpuses.append(corpus)
                            sentences.append(sentence)
                            original_sentences.append(original_sentence)
                            vectors.append(vector)
        except:
            break

In [5]:
df_dev = pd.DataFrame(list(zip(corpuses, id_nums, ts, original_sentences, vectors)),
               columns =['cor', 'ids', 'class', 'text', 'labels'])
df_dev = df_dev.set_index('ids')

In [6]:
# Test set

address = './Dataset/Test.PubTator'

with open(address) as f:
    
    id_nums = list()
    ts = list()
    titles = list()
    abstracts = list()
    original_texts = list()
    texts = list()
    results = list()
    original_results = list()
    vectors = list()
    corpuses = list()
    sentences = list()
    original_sentences = list()
    
    for x in f.read().split('\n\n'):
        
        try:
            id_num = x.split('|t|')[0]
            if id_num in ['15485686', '15818664', '19276632', '28846666', '24442316', '18507837']:
                continue
            else:
                id_num = id_num
            
            title = x.split('|t|')[1].split('\n')[0].rstrip()
            if title.endswith('?'):
                title = title[:-1] + '.'
            elif title.endswith('.'):
                title = title
            else:
                title = title + '.'
                
            abstract = x.split('|a|')[1].split('\n')[0].rstrip()
            if abstract.endswith('..'):
                abstract = abstract[:-1]
            elif abstract.endswith('.'):
                abstract = abstract
            else:
                abstract = abstract + '.'

            original_text = title + " " + abstract
            
            tags = list()
            for y in x.split('\n')[2:]:
                pocetni = y.split('\t')[1]
                if pocetni.isdigit():
                    tag = y.split('\t')[4]
                    tags.append(tag)
            
            for t in set(tags):
                text = original_text
                pomeraj = 0
                for y in x.split('\n')[2:]:
                    pocetni = y.split('\t')[1]
                    if pocetni.isdigit():
                        pocetni = int(pocetni) - pomeraj
                        krajnji = int(y.split('\t')[2]) - pomeraj
                        NER, tag = y.split('\t')[3:5]
                        
                        if tag == t:
                            NER_len = len(WordPunctTokenizer().tokenize(NER))
                            if NER_len == 1:
                                a = 'xxxxx'
                            else:
                                a = 'xxxxx'+' yyyyy'*(NER_len-1)
                        
                            text = text[:pocetni] + a + text[krajnji:]
                        
                            pomeraj += krajnji - pocetni - len(a)
                    else:
                        continue
                result = WordPunctTokenizer().tokenize(text)
                original_result = WordPunctTokenizer().tokenize(original_text)
                if len(result)==len(original_result):
                    pass
                else:
                    print(id_num, 'NIJE OK', len(result), len(original_result))
                
                n = 0
                for m in original_result:
                    if m == '.' or m == ').':
                        k = original_result.index(m, n)
                        sentence = result[n:k+1]
                        original_sentence = original_result[n:k+1]
                        n = k + 1
                        if len(sentence)==len(original_sentence):
                            pass
                        else:
                            print(id_num, 'NIJE OK', sentence, original_sentence)
                        vector = list()
                        for i in sentence:
                            if i=='xxxxx':
                                i=1
                            elif i=='yyyyy':
                                i=1
                            else:
                                i=0
                            vector.append(i)
                        corpus = 'BioRED'
                        if len(vector) >= 3:
                            id_nums.append(id_num)
                            ts.append(t)
                            titles.append(title)
                            abstracts.append(abstract)
                            original_texts.append(original_text)
                            texts.append(text)
                            results.append(result)
                            original_results.append(original_result)
                            corpuses.append(corpus)
                            sentences.append(sentence)
                            original_sentences.append(original_sentence)
                            vectors.append(vector)
        except:
            break

In [7]:
df_test = pd.DataFrame(list(zip(corpuses, id_nums, ts, original_sentences, vectors)),
               columns =['cor', 'ids', 'class', 'text', 'labels'])
df_test = df_test.set_index('ids')

In [8]:
df = pd.concat([df_train, df_dev, df_test])

In [9]:
df['class'] = df['class'].apply(lambda x: x.replace('CellLine', 'Cell Line'))
df['class'] = df['class'].apply(lambda x: x.replace('ChemicalEntity', 'Chemical Entity'))
df['class'] = df['class'].apply(lambda x: x.replace('DiseaseOrPhenotypicFeature', 'Disease Or Phenotypic Feature'))
df['class'] = df['class'].apply(lambda x: x.replace('GeneOrGeneProduct', 'Gene Or Gene Product'))
df['class'] = df['class'].apply(lambda x: x.replace('OrganismTaxon', 'Organism Taxon'))
df['class'] = df['class'].apply(lambda x: x.replace('SequenceVariant', 'Sequence Variant'))
df['class'] = df['class'].apply(lambda x: x.split(' '))

In [10]:
df['labels'] = df['labels'].apply(lambda x: np.array(x))
df['labels'] = df['labels'].apply(lambda x: x.astype(np.int64))

In [11]:
df

,cor,class,text,labels
ids,,,,
10491763,BioRED,"[Sequence, Variant]","[Hepatocyte, nuclear, factor, -, 6, :, associa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10491763,BioRED,"[Sequence, Variant]","[The, transcription, factor, hepatocyte, nucle...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10491763,BioRED,"[Sequence, Variant]","[We, therefore, tested, the, hypothesis, that,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10491763,BioRED,"[Sequence, Variant]","[We, cloned, the, coding, region, as, well, as...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10491763,BioRED,"[Sequence, Variant]","[We, then, examined, them, on, genomic, DNA, i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
24840785,BioRED,"[Chemical, Entity]","[The, results, showed, that, aconitine, stimul...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
24840785,BioRED,"[Chemical, Entity]","[The, expression, analysis, of, Ca, (, 2, +), ...","[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, ..."
24840785,BioRED,"[Chemical, Entity]","[The, expression, analysis, of, apoptosis, -, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
df = df.reset_index()
df

,ids,cor,class,text,labels
0,10491763,BioRED,"[Sequence, Variant]","[Hepatocyte, nuclear, factor, -, 6, :, associa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10491763,BioRED,"[Sequence, Variant]","[The, transcription, factor, hepatocyte, nucle...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10491763,BioRED,"[Sequence, Variant]","[We, therefore, tested, the, hypothesis, that,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,10491763,BioRED,"[Sequence, Variant]","[We, cloned, the, coding, region, as, well, as...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10491763,BioRED,"[Sequence, Variant]","[We, then, examined, them, on, genomic, DNA, i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
29895,24840785,BioRED,"[Chemical, Entity]","[The, results, showed, that, aconitine, stimul...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
29896,24840785,BioRED,"[Chemical, Entity]","[The, expression, analysis, of, Ca, (, 2, +), ...","[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, ..."
29897,24840785,BioRED,"[Chemical, Entity]","[The, expression, analysis, of, apoptosis, -, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
29898,24840785,BioRED,"[Chemical, Entity]","[Furthermore, ,, increased, phosphorylation, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
# using protocol 4 for dumping a .pkl file

filename = './BioRED_protokol4.pkl'
outfile = open(filename,'wb')
pickle.dump(df,outfile, protocol=4)

In [13]:
df.to_pickle('./BioRED_ivi.pkl')